<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled44.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# --- 1. Environment Class ---
class Environment:
    def __init__(self, total_sim_steps, days_per_cycle=50):
        self.total_sim_steps, self.days_per_cycle, self.time = total_sim_steps, days_per_cycle, 0
        num_years = total_sim_steps / (365 * days_per_cycle)
        daily_uv_cycle = np.sin(np.linspace(0, 2 * np.pi, days_per_cycle)); daily_uv_cycle[daily_uv_cycle < 0] = 0
        seasonal_amplitude_mod = 0.225 * np.sin(np.linspace(0, 2 * np.pi * num_years, total_sim_steps)) + 0.725
        self.uv_intensity = np.array([daily_uv_cycle[i % days_per_cycle] * seasonal_amplitude_mod[i] for i in range(total_sim_steps)])
        self.external_ntp_pool, self.external_aa_pool, self.external_trp_tyr_cys_pool = 20000.0, 20000.0, 500.0

    def get_current_uv(self):
        return self.uv_intensity[self.time] if self.time < self.total_sim_steps else 0.0
    def step(self):
        self.time += 1

# --- 2. Proto-Cell Class (Retuned for Extreme Punishment) ---
class ProtoCell:
    def __init__(self, env, strategy='cautious'):
        self.env, self.strategy = env, strategy
        self.rna_pool, self.dna_pool, self.atp_pool = 100.0, 10.0, 50.0
        self.mcyw_peptide_pool, self.other_peptide_pool = 5.0, 2.0
        self.internal_ntp_stock, self.internal_aa_stock, self.internal_trp_tyr_cys_stock = 100.0, 100.0, 10.0
        self.rna_damage_level, self.protein_synthesis_active, self.rna_dna_copying_active = 0.0, True, True
        self.uv_safety_status, self.premature_stop_signal = "SAFE", 0.0
        self.mcyw_synthesis_bias = 0.0 if strategy == 'generic' else 0.75

        # --- MORE PUNISHING PARAMETERS ---
        self.uv_damage_rate_per_uv = 0.80      # INCREASED: Extremely high damage
        self.repair_rate = 0.3                 # DECREASED: Repair is slow and inefficient
        self.trp_tyr_cys_for_repair_needed = 0.09 # INCREASED: Repair is more costly

        # Other parameters kept for sensitivity and shielding
        self.premature_stop_signal_threshold = 0.15
        self.precarious_codon_rate_per_damage = 1.5
        self.uv_protection_factor = 0.01
        self.damage_tolerance_threshold = 4.0
        self.atp_generation_rate_per_mcyw_uv=0.15; self.repair_atp_cost=0.007; self.foraging_efficiency_mcyw=0.20
        self.foraging_atp_cost=0.003; self.trp_tyr_cys_foraging_bias=10.0; self.protein_synthesis_rate=0.18
        self.protein_synthesis_ntp_cost=0.1; self.protein_synthesis_aa_cost=0.1; self.protein_synthesis_atp_cost=0.02
        self.rna_dna_copy_rate=0.05; self.rna_dna_copy_ntp_cost=0.05; self.rna_dna_copy_atp_cost=0.01
        self.rna_degradation_rate=0.0008; self.peptide_degradation_rate=0.0015; self.atp_degradation_rate=0.003

    def _apply_uv_damage(self):
        uv = self.env.get_current_uv()
        if uv > 0:
            protection = self.mcyw_peptide_pool * self.uv_protection_factor
            effective_uv = uv * max(0, 1 - protection)
            damage_increase = effective_uv * self.uv_damage_rate_per_uv
            self.rna_damage_level += damage_increase
            self.premature_stop_signal = damage_increase * self.precarious_codon_rate_per_damage
        else: self.premature_stop_signal = 0

    def _manage_state(self):
        stop_signal = self.premature_stop_signal
        if self.strategy == 'readthrough': stop_signal = 0
        norm_dmg = min(1., self.rna_damage_level / self.damage_tolerance_threshold)
        scarcity = 1. - (self.internal_trp_tyr_cys_stock / (self.trp_tyr_cys_for_repair_needed * 5)) if self.internal_trp_tyr_cys_stock < (self.trp_tyr_cys_for_repair_needed * 5) and self.env.get_current_uv() > 0.1 else 0.
        danger = (norm_dmg * 0.4) + (max(0, min(1, scarcity)) * 0.1) + (stop_signal * 0.5) # Re-weighted to heavily prioritize stop signal and damage
        if danger >= self.premature_stop_signal_threshold:
            self.protein_synthesis_active, self.rna_dna_copying_active, self.uv_safety_status = False, False, "ARRESTED"
        elif danger < self.premature_stop_signal_threshold * 0.5:
            self.protein_synthesis_active, self.rna_dna_copying_active, self.uv_safety_status = True, True, ("SAFE" if danger < self.premature_stop_signal_threshold * 0.1 and self.internal_trp_tyr_cys_stock > self.trp_tyr_cys_for_repair_needed * 10 else "CAUTIOUS")
        else: self.uv_safety_status = "CAUTIOUS"

    # Other methods are unchanged and condensed for brevity
    def _forage(self):
        if self.mcyw_peptide_pool>0 and self.atp_pool>self.foraging_atp_cost: self.atp_pool-=self.foraging_atp_cost*self.mcyw_peptide_pool; foraged=self.foraging_efficiency_mcyw*self.mcyw_peptide_pool; self.internal_ntp_stock+=min(foraged/2,self.env.external_ntp_pool); self.internal_aa_stock+=min(foraged/2,self.env.external_aa_pool); self.internal_trp_tyr_cys_stock+=min(foraged*self.trp_tyr_cys_foraging_bias,self.env.external_trp_tyr_cys_pool)
    def _synthesize_protein(self):
        if not self.protein_synthesis_active: return
        units=self.protein_synthesis_rate
        if not (self.internal_ntp_stock>=units*self.protein_synthesis_ntp_cost and self.internal_aa_stock>=units*self.protein_synthesis_aa_cost and self.atp_pool>=units*self.protein_synthesis_atp_cost): return
        m_units=0
        if self.mcyw_synthesis_bias>0 and self.internal_trp_tyr_cys_stock>0 and self.protein_synthesis_aa_cost>0: m_units=min(units*self.mcyw_synthesis_bias,self.internal_trp_tyr_cys_stock/(self.protein_synthesis_aa_cost*self.mcyw_synthesis_bias))
        o_units=units-m_units; self.mcyw_peptide_pool+=m_units; self.other_peptide_pool+=o_units
        if m_units>0: self.internal_trp_tyr_cys_stock-=m_units*(self.protein_synthesis_aa_cost*self.mcyw_synthesis_bias)
        self.internal_ntp_stock-=units*self.protein_synthesis_ntp_cost; self.internal_aa_stock-=units*self.protein_synthesis_aa_cost; self.atp_pool-=units*self.protein_synthesis_atp_cost
    def _synthesize_atp(self):
        if self.env.get_current_uv()>0 and self.mcyw_peptide_pool>0: self.atp_pool+=self.atp_generation_rate_per_mcyw_uv*self.mcyw_peptide_pool*self.env.get_current_uv()
    def _repair_rna_dna(self):
        if self.rna_damage_level > 0 and self.atp_pool>=self.repair_atp_cost and self.internal_trp_tyr_cys_stock>=self.trp_tyr_cys_for_repair_needed: self.atp_pool-=self.repair_atp_cost; self.internal_trp_tyr_cys_stock-=self.trp_tyr_cys_for_repair_needed; self.rna_damage_level-=min(self.rna_damage_level,self.repair_rate)
    def _copy_rna_dna(self):
        if not self.rna_dna_copying_active or not(self.internal_ntp_stock>=self.rna_dna_copy_ntp_cost and self.atp_pool>=self.rna_dna_copy_atp_cost): return
        self.internal_ntp_stock-=self.rna_dna_copy_ntp_cost; self.atp_pool-=self.rna_dna_copy_atp_cost; self.rna_pool+=self.rna_dna_copy_rate*0.9; self.dna_pool+=self.rna_dna_copy_rate*0.1
    def _degrade(self):
        for p in ['rna_pool','dna_pool','mcyw_peptide_pool','other_peptide_pool','atp_pool']: setattr(self,p,getattr(self,p)*(1-(self.rna_degradation_rate if 'rna' in p or 'dna' in p else(self.peptide_degradation_rate if 'peptide' in p else self.atp_degradation_rate))))
    def step(self):
        [getattr(self, p)() for p in ['_apply_uv_damage','_synthesize_atp','_forage','_repair_rna_dna','_manage_state','_synthesize_protein','_copy_rna_dna','_degrade']]

# --- 3. Simulation Runner & 4. Visualization ---
def run_and_plot():
    SIMULATION_DAYS = 730
    STEPS_PER_DAY = 50
    TOTAL_STEPS = SIMULATION_DAYS * STEPS_PER_DAY

    env=Environment(TOTAL_STEPS,STEPS_PER_DAY)
    cell_cautious,cell_generic,cell_readthrough=ProtoCell(env,strategy='cautious'),ProtoCell(env,strategy='generic'),ProtoCell(env,strategy='readthrough')

    for _ in range(TOTAL_STEPS):
        cell_cautious.step()
        cell_generic.step()
        cell_readthrough.step()
        env.step()

    # Prepare data for table
    final_data = {
        'Metric': ['RNA Pool', 'DNA Pool', 'ATP Pool', 'MCYW Peptide Pool', 'Other Peptide Pool', 'RNA Damage Level', 'UV Safety Status'],
        'Cautious': [cell_cautious.rna_pool, cell_cautious.dna_pool, cell_cautious.atp_pool, cell_cautious.mcyw_peptide_pool, cell_cautious.other_peptide_pool, cell_cautious.rna_damage_level, cell_cautious.uv_safety_status],
        'Generic': [cell_generic.rna_pool, cell_generic.dna_pool, cell_generic.atp_pool, cell_generic.mcyw_peptide_pool, cell_generic.other_peptide_pool, cell_generic.rna_damage_level, cell_generic.uv_safety_status],
        'Read-Through': [cell_readthrough.rna_pool, cell_readthrough.dna_pool, cell_readthrough.atp_pool, cell_readthrough.mcyw_peptide_pool, cell_readthrough.other_peptide_pool, cell_readthrough.rna_damage_level, cell_readthrough.uv_safety_status]
    }
    results_df = pd.DataFrame(final_data)
    display(results_df)


if __name__ == "__main__":
    run_and_plot()

,Metric,Cautious,Generic,Read-Through
0,RNA Pool,56.205,0.0,56.205
1,DNA Pool,6.245,0.0,6.245
2,ATP Pool,844.745795,0.0,844.745795
3,MCYW Peptide Pool,89.865,0.0,89.865
4,Other Peptide Pool,29.955,0.0,29.955
5,RNA Damage Level,0.0,5957.161498,0.0
6,UV Safety Status,SAFE,ARRESTED,SAFE


# Task
Modify the simulation code to capture the state of each proto-cell at each time step and save the entire simulation output to a single file in a machine-readable format (e.g., CSV or JSON).

## Modify protocell class

### Subtask:
Add a method to the `ProtoCell` class to capture its current state (all relevant attributes) as a dictionary.


**Reasoning**:
Add a method `get_state` to the `ProtoCell` class to capture the relevant attributes at the current time step.



In [ ]:
# --- 2. Proto-Cell Class (Retuned for Extreme Punishment) ---
class ProtoCell:
    def __init__(self, env, strategy='cautious'):
        self.env, self.strategy = env, strategy
        self.rna_pool, self.dna_pool, self.atp_pool = 100.0, 10.0, 50.0
        self.mcyw_peptide_pool, self.other_peptide_pool = 5.0, 2.0
        self.internal_ntp_stock, self.internal_aa_stock, self.internal_trp_tyr_cys_stock = 100.0, 100.0, 10.0
        self.rna_damage_level, self.protein_synthesis_active, self.rna_dna_copying_active = 0.0, True, True
        self.uv_safety_status, self.premature_stop_signal = "SAFE", 0.0
        self.mcyw_synthesis_bias = 0.0 if strategy == 'generic' else 0.75

        # --- MORE PUNISHING PARAMETERS ---
        self.uv_damage_rate_per_uv = 0.80      # INCREASED: Extremely high damage
        self.repair_rate = 0.3                 # DECREASED: Repair is slow and inefficient
        self.trp_tyr_cys_for_repair_needed = 0.09 # INCREASED: Repair is more costly

        # Other parameters kept for sensitivity and shielding
        self.premature_stop_signal_threshold = 0.15
        self.precarious_codon_rate_per_damage = 1.5
        self.uv_protection_factor = 0.01
        self.damage_tolerance_threshold = 4.0
        self.atp_generation_rate_per_mcyw_uv=0.15; self.repair_atp_cost=0.007; self.foraging_efficiency_mcyw=0.20
        self.foraging_atp_cost=0.003; self.trp_tyr_cys_foraging_bias=10.0; self.protein_synthesis_rate=0.18
        self.protein_synthesis_ntp_cost=0.1; self.protein_synthesis_aa_cost=0.1; self.protein_synthesis_atp_cost=0.02
        self.rna_dna_copy_rate=0.05; self.rna_dna_copy_ntp_cost=0.05; self.rna_dna_copy_atp_cost=0.01
        self.rna_degradation_rate=0.0008; self.peptide_degradation_rate=0.0015; self.atp_degradation_rate=0.003

    def _apply_uv_damage(self):
        uv = self.env.get_current_uv()
        if uv > 0:
            protection = self.mcyw_peptide_pool * self.uv_protection_factor
            effective_uv = uv * max(0, 1 - protection)
            damage_increase = effective_uv * self.uv_damage_rate_per_uv
            self.rna_damage_level += damage_increase
            self.premature_stop_signal = damage_increase * self.precarious_codon_rate_per_damage
        else: self.premature_stop_signal = 0

    def _manage_state(self):
        stop_signal = self.premature_stop_signal
        if self.strategy == 'readthrough': stop_signal = 0
        norm_dmg = min(1., self.rna_damage_level / self.damage_tolerance_threshold)
        scarcity = 1. - (self.internal_trp_tyr_cys_stock / (self.trp_tyr_cys_for_repair_needed * 5)) if self.internal_trp_tyr_cys_stock < (self.trp_tyr_cys_for_repair_needed * 5) and self.env.get_current_uv() > 0.1 else 0.
        danger = (norm_dmg * 0.4) + (max(0, min(1, scarcity)) * 0.1) + (stop_signal * 0.5) # Re-weighted to heavily prioritize stop signal and damage
        if danger >= self.premature_stop_signal_threshold:
            self.protein_synthesis_active, self.rna_dna_copying_active, self.uv_safety_status = False, False, "ARRESTED"
        elif danger < self.premature_stop_signal_threshold * 0.5:
            self.protein_synthesis_active, self.rna_dna_copying_active, self.uv_safety_status = True, True, ("SAFE" if danger < self.premature_stop_signal_threshold * 0.1 and self.internal_trp_tyr_cys_stock > self.trp_tyr_cys_for_repair_needed * 10 else "CAUTIOUS")
        else: self.uv_safety_status = "CAUTIOUS"

    # Other methods are unchanged and condensed for brevity
    def _forage(self):
        if self.mcyw_peptide_pool>0 and self.atp_pool>self.foraging_atp_cost: self.atp_pool-=self.foraging_atp_cost*self.mcyw_peptide_pool; foraged=self.foraging_efficiency_mcyw*self.mcyw_peptide_pool; self.internal_ntp_stock+=min(foraged/2,self.env.external_ntp_pool); self.internal_aa_stock+=min(foraged/2,self.env.external_aa_pool); self.internal_trp_tyr_cys_stock+=min(foraged*self.trp_tyr_cys_foraging_bias,self.env.external_trp_tyr_cys_pool)
    def _synthesize_protein(self):
        if not self.protein_synthesis_active: return
        units=self.protein_synthesis_rate
        if not (self.internal_ntp_stock>=units*self.protein_synthesis_ntp_cost and self.internal_aa_stock>=units*self.protein_synthesis_aa_cost and self.atp_pool>=units*self.protein_synthesis_atp_cost): return
        m_units=0
        if self.mcyw_synthesis_bias>0 and self.internal_trp_tyr_cys_stock>0 and self.protein_synthesis_aa_cost>0: m_units=min(units*self.mcyw_synthesis_bias,self.internal_trp_tyr_cys_stock/(self.protein_synthesis_aa_cost*self.mcyw_synthesis_bias))
        o_units=units-m_units; self.mcyw_peptide_pool+=m_units; self.other_peptide_pool+=o_units
        if m_units>0: self.internal_trp_tyr_cys_stock-=m_units*(self.protein_synthesis_aa_cost*self.mcyw_synthesis_bias)
        self.internal_ntp_stock-=units*self.protein_synthesis_ntp_cost; self.internal_aa_stock-=units*self.protein_synthesis_aa_cost; self.atp_pool-=units*self.protein_synthesis_atp_cost
    def _synthesize_atp(self):
        if self.env.get_current_uv()>0 and self.mcyw_peptide_pool>0: self.atp_pool+=self.atp_generation_rate_per_mcyw_uv*self.mcyw_peptide_pool*self.env.get_current_uv()
    def _repair_rna_dna(self):
        if self.rna_damage_level > 0 and self.atp_pool>=self.repair_atp_cost and self.internal_trp_tyr_cys_stock>=self.trp_tyr_cys_for_repair_needed: self.atp_pool-=self.repair_atp_cost; self.internal_trp_tyr_cys_stock-=self.trp_tyr_cys_for_repair_needed; self.rna_damage_level-=min(self.rna_damage_level,self.repair_rate)
    def _copy_rna_dna(self):
        if not self.rna_dna_copying_active or not(self.internal_ntp_stock>=self.rna_dna_copy_ntp_cost and self.atp_pool>=self.rna_dna_copy_atp_cost): return
        self.internal_ntp_stock-=self.rna_dna_copy_ntp_cost; self.atp_pool-=self.rna_dna_copy_atp_cost; self.rna_pool+=self.rna_dna_copy_rate*0.9; self.dna_pool+=self.rna_dna_copy_rate*0.1
    def _degrade(self):
        for p in ['rna_pool','dna_pool','mcyw_peptide_pool','other_peptide_pool','atp_pool']: setattr(self,p,getattr(self,p)*(1-(self.rna_degradation_rate if 'rna' in p or 'dna' in p else(self.peptide_degradation_rate if 'peptide' in p else self.atp_degradation_rate))))
    def step(self):
        [getattr(self, p)() for p in ['_apply_uv_damage','_synthesize_atp','_forage','_repair_rna_dna','_manage_state','_synthesize_protein','_copy_rna_dna','_degrade']]

    def get_state(self):
        """Captures the current state of the proto-cell as a dictionary."""
        state = {
            'time_step': self.env.time,
            'strategy': self.strategy,
            'rna_pool': self.rna_pool,
            'dna_pool': self.dna_pool,
            'atp_pool': self.atp_pool,
            'mcyw_peptide_pool': self.mcyw_peptide_pool,
            'other_peptide_pool': self.other_peptide_pool,
            'rna_damage_level': self.rna_damage_level,
            'uv_safety_status': self.uv_safety_status,
            'internal_ntp_stock': self.internal_ntp_stock,
            'internal_aa_stock': self.internal_aa_stock,
            'internal_trp_tyr_cys_stock': self.internal_trp_tyr_cys_stock,
            'protein_synthesis_active': self.protein_synthesis_active,
            'rna_dna_copying_active': self.rna_dna_copying_active,
            'premature_stop_signal': self.premature_stop_signal
        }
        return state

# --- 3. Simulation Runner & 4. Visualization ---
def run_and_plot():
    SIMULATION_DAYS = 730
    STEPS_PER_DAY = 50
    TOTAL_STEPS = SIMULATION_DAYS * STEPS_PER_DAY

    env=Environment(TOTAL_STEPS,STEPS_PER_DAY)
    cell_cautious,cell_generic,cell_readthrough=ProtoCell(env,strategy='cautious'),ProtoCell(env,strategy='generic'),ProtoCell(env,strategy='readthrough')

    for _ in range(TOTAL_STEPS):
        cell_cautious.step()
        cell_generic.step()
        cell_readthrough.step()
        env.step()

    # Prepare data for table
    final_data = {
        'Metric': ['RNA Pool', 'DNA Pool', 'ATP Pool', 'MCYW Peptide Pool', 'Other Peptide Pool', 'RNA Damage Level', 'UV Safety Status'],
        'Cautious': [cell_cautious.rna_pool, cell_cautious.dna_pool, cell_cautious.atp_pool, cell_cautious.mcyw_peptide_pool, cell_cautious.other_peptide_pool, cell_cautious.rna_damage_level, cell_cautious.uv_safety_status],
        'Generic': [cell_generic.rna_pool, cell_generic.dna_pool, cell_generic.atp_pool, cell_generic.mcyw_peptide_pool, cell_generic.other_peptide_pool, cell_generic.rna_damage_level, cell_generic.uv_safety_status],
        'Read-Through': [cell_readthrough.rna_pool, cell_readthrough.dna_pool, cell_readthrough.atp_pool, cell_readthrough.mcyw_peptide_pool, cell_readthrough.other_peptide_pool, cell_readthrough.rna_damage_level, cell_readthrough.uv_safety_status]
    }
    results_df = pd.DataFrame(final_data)
    display(results_df)


if __name__ == "__main__":
    run_and_plot()

,Metric,Cautious,Generic,Read-Through
0,RNA Pool,56.205,0.0,56.205
1,DNA Pool,6.245,0.0,6.245
2,ATP Pool,844.745795,0.0,844.745795
3,MCYW Peptide Pool,89.865,0.0,89.865
4,Other Peptide Pool,29.955,0.0,29.955
5,RNA Damage Level,0.0,5957.161498,0.0
6,UV Safety Status,SAFE,ARRESTED,SAFE


## Modify simulation runner

### Subtask:
Initialize lists to store the state of each proto-cell at each time step. In the simulation loop, after each step, call the state-capturing method for each proto-cell and append the results to the respective lists.


**Reasoning**:
Initialize empty lists to store the state of each proto-cell at each time step and append the state to the respective lists within the simulation loop.



In [ ]:
# --- 1. Environment Class ---
class Environment:
    def __init__(self, total_sim_steps, days_per_cycle=50):
        self.total_sim_steps, self.days_per_cycle, self.time = total_sim_steps, days_per_cycle, 0
        num_years = total_sim_steps / (365 * days_per_cycle)
        daily_uv_cycle = np.sin(np.linspace(0, 2 * np.pi, days_per_cycle)); daily_uv_cycle[daily_uv_cycle < 0] = 0
        seasonal_amplitude_mod = 0.225 * np.sin(np.linspace(0, 2 * np.pi * num_years, total_sim_steps)) + 0.725
        self.uv_intensity = np.array([daily_uv_cycle[i % days_per_cycle] * seasonal_amplitude_mod[i] for i in range(total_sim_steps)])
        self.external_ntp_pool, self.external_aa_pool, self.external_trp_tyr_cys_pool = 20000.0, 20000.0, 500.0

    def get_current_uv(self):
        return self.uv_intensity[self.time] if self.time < self.total_sim_steps else 0.0
    def step(self):
        self.time += 1

# --- 2. Proto-Cell Class (Retuned for Extreme Punishment) ---
class ProtoCell:
    def __init__(self, env, strategy='cautious'):
        self.env, self.strategy = env, strategy
        self.rna_pool, self.dna_pool, self.atp_pool = 100.0, 10.0, 50.0
        self.mcyw_peptide_pool, self.other_peptide_pool = 5.0, 2.0
        self.internal_ntp_stock, self.internal_aa_stock, self.internal_trp_tyr_cys_stock = 100.0, 100.0, 10.0
        self.rna_damage_level, self.protein_synthesis_active, self.rna_dna_copying_active = 0.0, True, True
        self.uv_safety_status, self.premature_stop_signal = "SAFE", 0.0
        self.mcyw_synthesis_bias = 0.0 if strategy == 'generic' else 0.75

        # --- MORE PUNISHING PARAMETERS ---
        self.uv_damage_rate_per_uv = 0.80      # INCREASED: Extremely high damage
        self.repair_rate = 0.3                 # DECREASED: Repair is slow and inefficient
        self.trp_tyr_cys_for_repair_needed = 0.09 # INCREASED: Repair is more costly

        # Other parameters kept for sensitivity and shielding
        self.premature_stop_signal_threshold = 0.15
        self.precarious_codon_rate_per_damage = 1.5
        self.uv_protection_factor = 0.01
        self.damage_tolerance_threshold = 4.0
        self.atp_generation_rate_per_mcyw_uv=0.15; self.repair_atp_cost=0.007; self.foraging_efficiency_mcyw=0.20
        self.foraging_atp_cost=0.003; self.trp_tyr_cys_foraging_bias=10.0; self.protein_synthesis_rate=0.18
        self.protein_synthesis_ntp_cost=0.1; self.protein_synthesis_aa_cost=0.1; self.protein_synthesis_atp_cost=0.02
        self.rna_dna_copy_rate=0.05; self.rna_dna_copy_ntp_cost=0.05; self.rna_dna_copy_atp_cost=0.01
        self.rna_degradation_rate=0.0008; self.peptide_degradation_rate=0.0015; self.atp_degradation_rate=0.003

    def _apply_uv_damage(self):
        uv = self.env.get_current_uv()
        if uv > 0:
            protection = self.mcyw_peptide_pool * self.uv_protection_factor
            effective_uv = uv * max(0, 1 - protection)
            damage_increase = effective_uv * self.uv_damage_rate_per_uv
            self.rna_damage_level += damage_increase
            self.premature_stop_signal = damage_increase * self.precarious_codon_rate_per_damage
        else: self.premature_stop_signal = 0

    def _manage_state(self):
        stop_signal = self.premature_stop_signal
        if self.strategy == 'readthrough': stop_signal = 0
        norm_dmg = min(1., self.rna_damage_level / self.damage_tolerance_threshold)
        scarcity = 1. - (self.internal_trp_tyr_cys_stock / (self.trp_tyr_cys_for_repair_needed * 5)) if self.internal_trp_tyr_cys_stock < (self.trp_tyr_cys_for_repair_needed * 5) and self.env.get_current_uv() > 0.1 else 0.
        danger = (norm_dmg * 0.4) + (max(0, min(1, scarcity)) * 0.1) + (stop_signal * 0.5) # Re-weighted to heavily prioritize stop signal and damage
        if danger >= self.premature_stop_signal_threshold:
            self.protein_synthesis_active, self.rna_dna_copying_active, self.uv_safety_status = False, False, "ARRESTED"
        elif danger < self.premature_stop_signal_threshold * 0.5:
            self.protein_synthesis_active, self.rna_dna_copying_active, self.uv_safety_status = True, True, ("SAFE" if danger < self.premature_stop_signal_threshold * 0.1 and self.internal_trp_tyr_cys_stock > self.trp_tyr_cys_for_repair_needed * 10 else "CAUTIOUS")
        else: self.uv_safety_status = "CAUTIOUS"

    # Other methods are unchanged and condensed for brevity
    def _forage(self):
        if self.mcyw_peptide_pool>0 and self.atp_pool>self.foraging_atp_cost: self.atp_pool-=self.foraging_atp_cost*self.mcyw_peptide_pool; foraged=self.foraging_efficiency_mcyw*self.mcyw_peptide_pool; self.internal_ntp_stock+=min(foraged/2,self.env.external_ntp_pool); self.internal_aa_stock+=min(foraged/2,self.env.external_aa_pool); self.internal_trp_tyr_cys_stock+=min(foraged*self.trp_tyr_cys_foraging_bias,self.env.external_trp_tyr_cys_pool)
    def _synthesize_protein(self):
        if not self.protein_synthesis_active: return
        units=self.protein_synthesis_rate
        if not (self.internal_ntp_stock>=units*self.protein_synthesis_ntp_cost and self.internal_aa_stock>=units*self.protein_synthesis_aa_cost and self.atp_pool>=units*self.protein_synthesis_atp_cost): return
        m_units=0
        if self.mcyw_synthesis_bias>0 and self.internal_trp_tyr_cys_stock>0 and self.protein_synthesis_aa_cost>0: m_units=min(units*self.mcyw_synthesis_bias,self.internal_trp_tyr_cys_stock/(self.protein_synthesis_aa_cost*self.mcyw_synthesis_bias))
        o_units=units-m_units; self.mcyw_peptide_pool+=m_units; self.other_peptide_pool+=o_units
        if m_units>0: self.internal_trp_tyr_cys_stock-=m_units*(self.protein_synthesis_aa_cost*self.mcyw_synthesis_bias)
        self.internal_ntp_stock-=units*self.protein_synthesis_ntp_cost; self.internal_aa_stock-=units*self.protein_synthesis_aa_cost; self.atp_pool-=units*self.protein_synthesis_atp_cost
    def _synthesize_atp(self):
        if self.env.get_current_uv()>0 and self.mcyw_peptide_pool>0: self.atp_pool+=self.atp_generation_rate_per_mcyw_uv*self.mcyw_peptide_pool*self.env.get_current_uv()
    def _repair_rna_dna(self):
        if self.rna_damage_level > 0 and self.atp_pool>=self.repair_atp_cost and self.internal_trp_tyr_cys_stock>=self.trp_tyr_cys_for_repair_needed: self.atp_pool-=self.repair_atp_cost; self.internal_trp_tyr_cys_stock-=self.trp_tyr_cys_for_repair_needed; self.rna_damage_level-=min(self.rna_damage_level,self.repair_rate)
    def _copy_rna_dna(self):
        if not self.rna_dna_copying_active or not(self.internal_ntp_stock>=self.rna_dna_copy_ntp_cost and self.atp_pool>=self.rna_dna_copy_atp_cost): return
        self.internal_ntp_stock-=self.rna_dna_copy_ntp_cost; self.atp_pool-=self.rna_dna_copy_atp_cost; self.rna_pool+=self.rna_dna_copy_rate*0.9; self.dna_pool+=self.rna_dna_copy_rate*0.1
    def _degrade(self):
        for p in ['rna_pool','dna_pool','mcyw_peptide_pool','other_peptide_pool','atp_pool']: setattr(self,p,getattr(self,p)*(1-(self.rna_degradation_rate if 'rna' in p or 'dna' in p else(self.peptide_degradation_rate if 'peptide' in p else self.atp_degradation_rate))))
    def step(self):
        [getattr(self, p)() for p in ['_apply_uv_damage','_synthesize_atp','_forage','_repair_rna_dna','_manage_state','_synthesize_protein','_copy_rna_dna','_degrade']]

    def get_state(self):
        """Captures the current state of the proto-cell as a dictionary."""
        state = {
            'time_step': self.env.time,
            'strategy': self.strategy,
            'rna_pool': self.rna_pool,
            'dna_pool': self.dna_pool,
            'atp_pool': self.atp_pool,
            'mcyw_peptide_pool': self.mcyw_peptide_pool,
            'other_peptide_pool': self.other_peptide_pool,
            'rna_damage_level': self.rna_damage_level,
            'uv_safety_status': self.uv_safety_status,
            'internal_ntp_stock': self.internal_ntp_stock,
            'internal_aa_stock': self.internal_aa_stock,
            'internal_trp_tyr_cys_stock': self.internal_trp_tyr_cys_stock,
            'protein_synthesis_active': self.protein_synthesis_active,
            'rna_dna_copying_active': self.rna_dna_copying_active,
            'premature_stop_signal': self.premature_stop_signal
        }
        return state

# --- 3. Simulation Runner & 4. Visualization ---
def run_and_plot():
    SIMULATION_DAYS = 730
    STEPS_PER_DAY = 50
    TOTAL_STEPS = SIMULATION_DAYS * STEPS_PER_DAY

    env=Environment(TOTAL_STEPS,STEPS_PER_DAY)
    cell_cautious,cell_generic,cell_readthrough=ProtoCell(env,strategy='cautious'),ProtoCell(env,strategy='generic'),ProtoCell(env,strategy='readthrough')

    # Initialize lists to store states
    cautious_states = []
    generic_states = []
    readthrough_states = []

    for _ in range(TOTAL_STEPS):
        cell_cautious.step()
        cell_generic.step()
        cell_readthrough.step()
        env.step()

        # Capture and append states
        cautious_states.append(cell_cautious.get_state())
        generic_states.append(cell_generic.get_state())
        readthrough_states.append(cell_readthrough.get_state())


    # Prepare data for table
    final_data = {
        'Metric': ['RNA Pool', 'DNA Pool', 'ATP Pool', 'MCYW Peptide Pool', 'Other Peptide Pool', 'RNA Damage Level', 'UV Safety Status'],
        'Cautious': [cell_cautious.rna_pool, cell_cautious.dna_pool, cell_cautious.atp_pool, cell_cautious.mcyw_peptide_pool, cell_cautious.other_peptide_pool, cell_cautious.rna_damage_level, cell_cautious.uv_safety_status],
        'Generic': [cell_generic.rna_pool, cell_generic.dna_pool, cell_generic.atp_pool, cell_generic.mcyw_peptide_pool, cell_generic.other_peptide_pool, cell_generic.rna_damage_level, cell_generic.uv_safety_status],
        'Read-Through': [cell_readthrough.rna_pool, cell_readthrough.dna_pool, cell_readthrough.atp_pool, cell_readthrough.mcyw_peptide_pool, cell_readthrough.other_peptide_pool, cell_readthrough.rna_damage_level, cell_readthrough.uv_safety_status]
    }
    results_df = pd.DataFrame(final_data)
    display(results_df)


if __name__ == "__main__":
    run_and_plot()

,Metric,Cautious,Generic,Read-Through
0,RNA Pool,56.205,0.0,56.205
1,DNA Pool,6.245,0.0,6.245
2,ATP Pool,844.745795,0.0,844.745795
3,MCYW Peptide Pool,89.865,0.0,89.865
4,Other Peptide Pool,29.955,0.0,29.955
5,RNA Damage Level,0.0,5957.161498,0.0
6,UV Safety Status,SAFE,ARRESTED,SAFE


## Organize and store data

### Subtask:
After the simulation, combine the collected state data for all proto-cell strategies into a single data structure (e.g., a list of dictionaries or a nested dictionary). Save the collected data to a file (e.g., a CSV or JSON file) for later analysis.


**Reasoning**:
Combine the collected state data for all proto-cell strategies into a single pandas DataFrame and save it to a CSV file.



In [ ]:
# --- 1. Environment Class ---
class Environment:
    def __init__(self, total_sim_steps, days_per_cycle=50):
        self.total_sim_steps, self.days_per_cycle, self.time = total_sim_steps, days_per_cycle, 0
        num_years = total_sim_steps / (365 * days_per_cycle)
        daily_uv_cycle = np.sin(np.linspace(0, 2 * np.pi, days_per_cycle)); daily_uv_cycle[daily_uv_cycle < 0] = 0
        seasonal_amplitude_mod = 0.225 * np.sin(np.linspace(0, 2 * np.pi * num_years, total_sim_steps)) + 0.725
        self.uv_intensity = np.array([daily_uv_cycle[i % days_per_cycle] * seasonal_amplitude_mod[i] for i in range(total_sim_steps)])
        self.external_ntp_pool, self.external_aa_pool, self.external_trp_tyr_cys_pool = 20000.0, 20000.0, 500.0

    def get_current_uv(self):
        return self.uv_intensity[self.time] if self.time < self.total_sim_steps else 0.0
    def step(self):
        self.time += 1

# --- 2. Proto-Cell Class (Retuned for Extreme Punishment) ---
class ProtoCell:
    def __init__(self, env, strategy='cautious'):
        self.env, self.strategy = env, strategy
        self.rna_pool, self.dna_pool, self.atp_pool = 100.0, 10.0, 50.0
        self.mcyw_peptide_pool, self.other_peptide_pool = 5.0, 2.0
        self.internal_ntp_stock, self.internal_aa_stock, self.internal_trp_tyr_cys_stock = 100.0, 100.0, 10.0
        self.rna_damage_level, self.protein_synthesis_active, self.rna_dna_copying_active = 0.0, True, True
        self.uv_safety_status, self.premature_stop_signal = "SAFE", 0.0
        self.mcyw_synthesis_bias = 0.0 if strategy == 'generic' else 0.75

        # --- MORE PUNISHING PARAMETERS ---
        self.uv_damage_rate_per_uv = 0.80      # INCREASED: Extremely high damage
        self.repair_rate = 0.3                 # DECREASED: Repair is slow and inefficient
        self.trp_tyr_cys_for_repair_needed = 0.09 # INCREASED: Repair is more costly

        # Other parameters kept for sensitivity and shielding
        self.premature_stop_signal_threshold = 0.15
        self.precarious_codon_rate_per_damage = 1.5
        self.uv_protection_factor = 0.01
        self.damage_tolerance_threshold = 4.0
        self.atp_generation_rate_per_mcyw_uv=0.15; self.repair_atp_cost=0.007; self.foraging_efficiency_mcyw=0.20
        self.foraging_atp_cost=0.003; self.trp_tyr_cys_foraging_bias=10.0; self.protein_synthesis_rate=0.18
        self.protein_synthesis_ntp_cost=0.1; self.protein_synthesis_aa_cost=0.1; self.protein_synthesis_atp_cost=0.02
        self.rna_dna_copy_rate=0.05; self.rna_dna_copy_ntp_cost=0.05; self.rna_dna_copy_atp_cost=0.01
        self.rna_degradation_rate=0.0008; self.peptide_degradation_rate=0.0015; self.atp_degradation_rate=0.003

    def _apply_uv_damage(self):
        uv = self.env.get_current_uv()
        if uv > 0:
            protection = self.mcyw_peptide_pool * self.uv_protection_factor
            effective_uv = uv * max(0, 1 - protection)
            damage_increase = effective_uv * self.uv_damage_rate_per_uv
            self.rna_damage_level += damage_increase
            self.premature_stop_signal = damage_increase * self.precarious_codon_rate_per_damage
        else: self.premature_stop_signal = 0

    def _manage_state(self):
        stop_signal = self.premature_stop_signal
        if self.strategy == 'readthrough': stop_signal = 0
        norm_dmg = min(1., self.rna_damage_level / self.damage_tolerance_threshold)
        scarcity = 1. - (self.internal_trp_tyr_cys_stock / (self.trp_tyr_cys_for_repair_needed * 5)) if self.internal_trp_tyr_cys_stock < (self.trp_tyr_cys_for_repair_needed * 5) and self.env.get_current_uv() > 0.1 else 0.
        danger = (norm_dmg * 0.4) + (max(0, min(1, scarcity)) * 0.1) + (stop_signal * 0.5) # Re-weighted to heavily prioritize stop signal and damage
        if danger >= self.premature_stop_signal_threshold:
            self.protein_synthesis_active, self.rna_dna_copying_active, self.uv_safety_status = False, False, "ARRESTED"
        elif danger < self.premature_stop_signal_threshold * 0.5:
            self.protein_synthesis_active, self.rna_dna_copying_active, self.uv_safety_status = True, True, ("SAFE" if danger < self.premature_stop_signal_threshold * 0.1 and self.internal_trp_tyr_cys_stock > self.trp_tyr_cys_for_repair_needed * 10 else "CAUTIOUS")
        else: self.uv_safety_status = "CAUTIOUS"

    # Other methods are unchanged and condensed for brevity
    def _forage(self):
        if self.mcyw_peptide_pool>0 and self.atp_pool>self.foraging_atp_cost: self.atp_pool-=self.foraging_atp_cost*self.mcyw_peptide_pool; foraged=self.foraging_efficiency_mcyw*self.mcyw_peptide_pool; self.internal_ntp_stock+=min(foraged/2,self.env.external_ntp_pool); self.internal_aa_stock+=min(foraged/2,self.env.external_aa_pool); self.internal_trp_tyr_cys_stock+=min(foraged*self.trp_tyr_cys_foraging_bias,self.env.external_trp_tyr_cys_pool)
    def _synthesize_protein(self):
        if not self.protein_synthesis_active: return
        units=self.protein_synthesis_rate
        if not (self.internal_ntp_stock>=units*self.protein_synthesis_ntp_cost and self.internal_aa_stock>=units*self.protein_synthesis_aa_cost and self.atp_pool>=units*self.protein_synthesis_atp_cost): return
        m_units=0
        if self.mcyw_synthesis_bias>0 and self.internal_trp_tyr_cys_stock>0 and self.protein_synthesis_aa_cost>0: m_units=min(units*self.mcyw_synthesis_bias,self.internal_trp_tyr_cys_stock/(self.protein_synthesis_aa_cost*self.mcyw_synthesis_bias))
        o_units=units-m_units; self.mcyw_peptide_pool+=m_units; self.other_peptide_pool+=o_units
        if m_units>0: self.internal_trp_tyr_cys_stock-=m_units*(self.protein_synthesis_aa_cost*self.mcyw_synthesis_bias)
        self.internal_ntp_stock-=units*self.protein_synthesis_ntp_cost; self.internal_aa_stock-=units*self.protein_synthesis_aa_cost; self.atp_pool-=units*self.protein_synthesis_atp_cost
    def _synthesize_atp(self):
        if self.env.get_current_uv()>0 and self.mcyw_peptide_pool>0: self.atp_pool+=self.atp_generation_rate_per_mcyw_uv*self.mcyw_peptide_pool*self.env.get_current_uv()
    def _repair_rna_dna(self):
        if self.rna_damage_level > 0 and self.atp_pool>=self.repair_atp_cost and self.internal_trp_tyr_cys_stock>=self.trp_tyr_cys_for_repair_needed: self.atp_pool-=self.repair_atp_cost; self.internal_trp_tyr_cys_stock-=self.trp_tyr_cys_for_repair_needed; self.rna_damage_level-=min(self.rna_damage_level,self.repair_rate)
    def _copy_rna_dna(self):
        if not self.rna_dna_copying_active or not(self.internal_ntp_stock>=self.rna_dna_copy_ntp_cost and self.atp_pool>=self.rna_dna_copy_atp_cost): return
        self.internal_ntp_stock-=self.rna_dna_copy_ntp_cost; self.atp_pool-=self.rna_dna_copy_atp_cost; self.rna_pool+=self.rna_dna_copy_rate*0.9; self.dna_pool+=self.rna_dna_copy_rate*0.1
    def _degrade(self):
        for p in ['rna_pool','dna_pool','mcyw_peptide_pool','other_peptide_pool','atp_pool']: setattr(self,p,getattr(self,p)*(1-(self.rna_degradation_rate if 'rna' in p or 'dna' in p else(self.peptide_degradation_rate if 'peptide' in p else self.atp_degradation_rate))))
    def step(self):
        [getattr(self, p)() for p in ['_apply_uv_damage','_synthesize_atp','_forage','_repair_rna_dna','_manage_state','_synthesize_protein','_copy_rna_dna','_degrade']]

    def get_state(self):
        """Captures the current state of the proto-cell as a dictionary."""
        state = {
            'time_step': self.env.time,
            'strategy': self.strategy,
            'rna_pool': self.rna_pool,
            'dna_pool': self.dna_pool,
            'atp_pool': self.atp_pool,
            'mcyw_peptide_pool': self.mcyw_peptide_pool,
            'other_peptide_pool': self.other_peptide_pool,
            'rna_damage_level': self.rna_damage_level,
            'uv_safety_status': self.uv_safety_status,
            'internal_ntp_stock': self.internal_ntp_stock,
            'internal_aa_stock': self.internal_aa_stock,
            'internal_trp_tyr_cys_stock': self.internal_trp_tyr_cys_stock,
            'protein_synthesis_active': self.protein_synthesis_active,
            'rna_dna_copying_active': self.rna_dna_copying_active,
            'premature_stop_signal': self.premature_stop_signal
        }
        return state

# --- 3. Simulation Runner & 4. Visualization ---
def run_and_plot():
    SIMULATION_DAYS = 730
    STEPS_PER_DAY = 50
    TOTAL_STEPS = SIMULATION_DAYS * STEPS_PER_DAY

    env=Environment(TOTAL_STEPS,STEPS_PER_DAY)
    cell_cautious,cell_generic,cell_readthrough=ProtoCell(env,strategy='cautious'),ProtoCell(env,strategy='generic'),ProtoCell(env,strategy='readthrough')

    # Initialize lists to store states
    cautious_states = []
    generic_states = []
    readthrough_states = []

    for _ in range(TOTAL_STEPS):
        cell_cautious.step()
        cell_generic.step()
        cell_readthrough.step()
        env.step()

        # Capture and append states
        cautious_states.append(cell_cautious.get_state())
        generic_states.append(cell_generic.get_state())
        readthrough_states.append(cell_readthrough.get_state())

    # Combine states into a single DataFrame
    all_states_df = pd.DataFrame(cautious_states + generic_states + readthrough_states)

    # Save the combined DataFrame to a CSV file
    all_states_df.to_csv('simulation_output.csv', index=False)

    # Prepare data for final table (optional - can be removed if not needed)
    final_data = {
        'Metric': ['RNA Pool', 'DNA Pool', 'ATP Pool', 'MCYW Peptide Pool', 'Other Peptide Pool', 'RNA Damage Level', 'UV Safety Status'],
        'Cautious': [cell_cautious.rna_pool, cell_cautious.dna_pool, cell_cautious.atp_pool, cell_cautious.mcyw_peptide_pool, cell_cautious.other_peptide_pool, cell_cautious.rna_damage_level, cell_cautious.uv_safety_status],
        'Generic': [cell_generic.rna_pool, cell_generic.dna_pool, cell_generic.atp_pool, cell_generic.mcyw_peptide_pool, cell_generic.other_peptide_pool, cell_generic.rna_damage_level, cell_generic.uv_safety_status],
        'Read-Through': [cell_readthrough.rna_pool, cell_readthrough.dna_pool, cell_readthrough.atp_pool, cell_readthrough.mcyw_peptide_pool, cell_readthrough.other_peptide_pool, cell_readthrough.rna_damage_level, cell_readthrough.uv_safety_status]
    }
    results_df = pd.DataFrame(final_data)
    display(results_df)


if __name__ == "__main__":
    run_and_plot()

,Metric,Cautious,Generic,Read-Through
0,RNA Pool,56.205,0.0,56.205
1,DNA Pool,6.245,0.0,6.245
2,ATP Pool,844.745795,0.0,844.745795
3,MCYW Peptide Pool,89.865,0.0,89.865
4,Other Peptide Pool,29.955,0.0,29.955
5,RNA Damage Level,0.0,5957.161498,0.0
6,UV Safety Status,SAFE,ARRESTED,SAFE


## Summary:

### Data Analysis Key Findings

*   The simulation successfully captured the state of each proto-cell (Cautious, Generic, and Read-Through) at every time step.
*   A method `get_state()` was added to the `ProtoCell` class to facilitate the capture of relevant attributes at each step.
*   The captured states for all strategies were combined into a single pandas DataFrame after the simulation.
*   The complete simulation output, including the state of each proto-cell at each time step, was successfully saved to a CSV file named `simulation_output.csv`.

### Insights or Next Steps

*   The saved `simulation_output.csv` file contains granular data that can be used for detailed analysis of how different proto-cell strategies evolve over time under the simulated environmental conditions, allowing for time-series analysis of key metrics.
*   Visualize the time-series data from the `simulation_output.csv` file to observe trends and compare the performance of the different strategies throughout the simulation period.
